<a href="https://colab.research.google.com/github/ulung3ko/text-anaytics-assignment-2-topic-modelling/blob/main/topic_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Identifikasi Masalah
## Latar Belakang
Dalam dunia akademik, artikel ilmiah diterbitkan setiap hari. Volume informasi ini menyulitkan akademisi baik itu mahasiswa, dosen, dan peneliti untuk mengikuti perkembangan terbaru secara menyeluruh. Disinilah penting nya topic modeling, dengan teknik ini, kita dapat secara otomatis mengelompokkan kumpulan abstrak artikel ilmiah ke dalam topik-topik utama seperti Artificial Intelligence, Cybersecurity, atau Computer Vision.

Topic modeling memungkinkan kita untuk:

- Mengidentifikasi tren penelitian yang sedang berkembang.
- Menemukan artikel relevan tanpa membaca satu per satu.
- Menyederhanakan eksplorasi literatur dalam jumlah besar.

Dalam tugas ini, kita akan membandingkan dua metode topic modeling:

- Latent Dirichlet Allocation (LDA).
- BERTopic.

Fokus tugas ini yaitu menilai model mana yang memberikan hasil pengelompokan topik yang lebih relevan, dapat dimengerti, dan sesuai dengan kategori asli artikel.

## Pertanyaan Penelitian
1. Topik apa saja yang berhasil ditemukan oleh masing-masing metode?

2. Berapa jumlah topik optimal, dan model mana yang memberikan skor kualitas topik terbaik?

3. Seberapa sesuai hasil topik dari masing-masing metode dengan label kategori asli dari artikel?



# Pengumpulan Dataset
## Sumber Data
Untuk tugas topic modeling ini, dataset yang digunakan adalah dataset publik "ArXiv Dataset" yang tersedia di platform Kaggle. Dataset ini berisi metadata dari jutaan publikasi ilmiah dari penerbit arXiv.org.

Setiap entri mencakup informasi seperti:
- id: ID unik artikel
- title: Judul artikel
- abstract: Ringkasan isi artikel (digunakan sebagai teks utama analisis)
- categories: Label bidang ilmu (misal: cs.AI, cs.CV, math.CO) yang akan digunakan untuk evaluasi model
- update_date: Tanggal terakhir artikel diperbarui

Dataset ini cocok dengan implementasi topic modeling karena berisi abstrak berkualitas tinggi, terstruktur, dan kaya informasi—ideal untuk pemrosesan bahasa alami (NLP).

Link Kaggle : https://www.kaggle.com/datasets/Cornell-University/arxiv

## Pengumpulan Data
Dataset asli berukuran besar (lebih dari 2 juta entri) dan disimpan dalam format JSON Lines (.jsonl), di mana setiap baris adalah satu objek JSON.

Supaya lebih relevan, dataset yang artikel yang digunakan hanya tahun 2021 keatas. Dari tahap ini, diperoleh DataFrame akhir (df_recent) dengan 1.055.586 baris. Karena keterbatasan komputasi, maka diambil sample sebanyak 100.00 saja.

In [1]:
# Ini merupakan kode dari kaggle sendiri untuk mendownload dataset nya
import kagglehub

# Download latest version
path = kagglehub.dataset_download("Cornell-University/arxiv")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/arxiv


In [2]:
"""Kode ini digunakan untuk memindahkan file dataset yang telah ter download
kedalam direktori '/content/dataset' google colab"""
import shutil
# Source folder dari kagglehub
src_path = "/kaggle/input/arxiv"

# Target folder (direktori kerja biasa)
dst_path = "/content/dataset"

# Salin semua isi folder
shutil.copytree(src_path, dst_path, dirs_exist_ok=True)

print("Dataset telah dipindahkan ke:", dst_path)

Dataset telah dipindahkan ke: /content/dataset


In [3]:
"""Kode dibawah digunakan untuk mengkonversi file dataset yang berformat json
ke dalam pandas dataframe, dan saat proses nya difilter hanya artikel atau paper
dengan tahun >= 2021"""

import pandas as pd
import json

data = []
file_path = '/content/dataset/arxiv-metadata-oai-snapshot.json'
start_year = 2021

with open(file_path, 'r') as f:
    for line in f:
        # Megubah setiap baris menjadi dictionary
        parsed_line = json.loads(line)

        # Membaca tahun dari kolom 'update_date'
        # try-except digunakan untuk antisipasi mana tau ada data yang aneh
        try:
            # Mendapatkan tahun dari kolom 'update date'
            year = int(parsed_line['update_date'][:4])

            # Simpa data >= 2021
            if year >= start_year:
                data.append(parsed_line)
        except (ValueError, TypeError):
            # Jika ada error maka dilanjutkan proses nya
            continue

# Membuat dataframe dari data-data yang sudah difilter
df = pd.DataFrame(data)

# Cek hasil dataframe
if not df.empty:
    print(f"Berhasil membaca {len(df)} baris data dari tahun {start_year} ke atas.")
    print("\nInformasi DataFrame:")
    df.info()

    print("\n5 baris pertama data terbaru:")
    print(df.head())
else:
    print(f"Tidak ada data yang ditemukan dari tahun {start_year} ke atas.")

Berhasil membaca 1055586 baris data dari tahun 2021 ke atas.

Informasi DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1055586 entries, 0 to 1055585
Data columns (total 14 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   id              1055586 non-null  object
 1   submitter       1055554 non-null  object
 2   authors         1055586 non-null  object
 3   title           1055586 non-null  object
 4   comments        673326 non-null   object
 5   journal-ref     208190 non-null   object
 6   doi             317727 non-null   object
 7   report-no       25292 non-null    object
 8   categories      1055586 non-null  object
 9   license         1054272 non-null  object
 10  abstract        1055586 non-null  object
 11  versions        1055586 non-null  object
 12  update_date     1055586 non-null  object
 13  authors_parsed  1055586 non-null  object
dtypes: object(14)
memory usage: 112.7+ MB

5 baris pertama data ter

In [4]:
"""Kode pada cell ini berfungsi untuk mengambil 100.000 data dari dataset
Hal ini dilakukan karena dataset terlalu besar dan tidak cukup komputasi."""

sample_size = 100000

print(f"\nUkuran DataFrame asli: {len(df)} baris.")

# Menyimpan 100.000 data ke df_sampled
df_sampled = df.sample(n=sample_size, random_state=42)

print(f"Ukuran DataFrame setelah di-sample: {len(df_sampled)} baris.")

# Timpa df asli dengan df_sampled (tidak digunakan lagi)
df = df_sampled.copy()

# Reset index DataFrame yang sudah dilakukan proses sample
df.reset_index(drop=True, inplace=True)

# Cek jumlah baris
df.shape


Ukuran DataFrame asli: 1055586 baris.
Ukuran DataFrame setelah di-sample: 100000 baris.


(100000, 14)

# Pra-pemrosesan Teks (pre-processing)
Proses ini dilakukan sesuai dengan intruksi soal.

In [5]:
# Kode dibawah adalah mendownload beberapa modul dari NLTK untu pre-processing teks
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [7]:
"""Kode ini digunakan untuk menggabungkan judul artikel dan abstrak menjadi
kolom teks untuk memperkaya informasi supaya LDA dan BertTopic lebih bisa
menangkap pola dan topik lebih baik"""

df['text'] = df['title'].fillna('') + ' ' + df['abstract'].fillna('')

In [8]:
"""Kode dibawah didunakan untuk mendefinisikan fungsi pre-processing
yang akan diaplikasikan pada dataset"""

import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Inisialisasi lemmatizer dan stopwords
lemmatizer = WordNetLemmatizer()
# Menggunakan stopwords berbahasa inggris karena dataset artikel berhasasa inggris
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Case Folding: Mengubah semua teks menjadi huruf kecil
    text = text.lower()

    # Filtering
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'[^a-z\s]', '', text)

    # Tokenisasi: Memecah teks menjadi token
    tokens = word_tokenize(text)

    # Stopword Removal & Lemmatization & Filtering kata pendek
    cleaned_tokens = []
    for word in tokens:
        # Cek apakah kata bukan stopword dan panjangnya lebih dari 2 huruf
        if word not in stop_words and len(word) > 2:
            # Lemmatization: Mengubah kata ke bentuk dasarnya
            cleaned_tokens.append(lemmatizer.lemmatize(word))

    return cleaned_tokens

In [9]:
# Test hasil fungsi pre-processing yang telah dibuat
print("Hasil uji coba pra-pemrosesan pada 5 baris pertama:")
contoh_hasil = df['text'].head(5).apply(preprocess_text)
print(contoh_hasil)

Hasil uji coba pra-pemrosesan pada 5 baris pertama:
0    [morphological, computing, logic, underlying, ...
1    [sixteen, point, mathbbp, inverse, galois, pro...
2    [aibased, aortic, vessel, tree, segmentation, ...
3    [pathwise, unique, solution, stochastic, avera...
4    [twodimensional, stabilized, discontinuous, ga...
Name: text, dtype: object


In [10]:
# Mengaplikasikan fungsi pada seluruh data di dataset dan disimpan pada kolom baru
df['processed_text'] = df['text'].apply(preprocess_text)

In [11]:
# Menampilkan dataset sebelum dan sesudah pre-processing
print(df[['text', 'processed_text']].sample(5))

                                                    text  \
32754  Simulation-Based Performance Evaluation of 3D ...   
57631  A Hybrid Framework for Statistical Feature Sel...   
1884   Disentangling Complex Systems: IdopNetwork Mee...   
36042  Puzzles in 3D Off-Shell Geometries via VTQFT  ...   
39890  Mirror Descent on Reproducing Kernel Banach Sp...   

                                          processed_text  
32754  [simulationbased, performance, evaluation, obj...  
57631  [hybrid, framework, statistical, feature, sele...  
1884   [disentangling, complex, system, idopnetwork, ...  
36042  [puzzle, offshell, geometry, via, vtqft, point...  
39890  [mirror, descent, reproducing, kernel, banach,...  


# Penerapan Algoritma Topic Modeling

## LDA
Penerapan algorima LDA pada tugas ini mengunakan library gensim, untuk dapat diaplikasikan pada LDA, kolom processed_text wajib diubah dalam bentuk dictionary lalu corpus. Saat mengkonversi kedalam dictionary, kata-kata dengan kemunculan kurang dari 15 dihapus dan jika muncul diatas 50% dari dokumen juga dihapus (dianggap umum), lalu setelah itu dijadikan corpus dengan metode BoW.

In [17]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 22.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
thinc 8.3.6 req

In [12]:
"""Kode cell ini berfungsi untuk membuat corpus dan dictionary, karena LDA
akan dijalankan dengan gensim yang membutuhkan corpus dan dictionary."""
from gensim.corpora import Dictionary
from gensim.models import LdaModel
import pandas as pd

documents = [doc for doc in df['processed_text'] if doc]

# Membuat Dictionary (memetakan setiap kata unik menjadi sebuah ID)
dictionary = Dictionary(documents)

# Memfilter kata eksrim, kata kuran dari 15 dan lebih dari 50% isi dokumen dihapus
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

# Membuat Corpus dengan Bag-of-Words
# BoW mengubah setiap dokumen menjadi representasi (ID kata, frekuensi)
corpus = [dictionary.doc2bow(doc) for doc in documents]

print(f"Jumlah kata unik dalam kamus setelah filtering: {len(dictionary)}")
print(f"Jumlah dokumen dalam corpus: {len(corpus)}")

Jumlah kata unik dalam kamus setelah filtering: 18248
Jumlah dokumen dalam corpus: 100000


In [13]:
# Latih model LDA
# Jumah topik awal adalah 10
NUM_TOPICS = 10

lda_model = LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=NUM_TOPICS,
    random_state=42,
    passes=10,
    alpha='auto',
    eta='auto'
)

In [14]:
# Menampilkan 10 kata kunci teratas untuk setiap topik
print("\nTopik yang ditemukan oleh LDA:")
topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)


Topik yang ditemukan oleh LDA:
(0, '0.026*"network" + 0.012*"system" + 0.011*"quantum" + 0.009*"performance" + 0.009*"communication" + 0.007*"design" + 0.007*"channel" + 0.007*"power" + 0.006*"device" + 0.006*"proposed"')
(1, '0.035*"graph" + 0.016*"number" + 0.015*"set" + 0.012*"bound" + 0.010*"show" + 0.009*"result" + 0.008*"problem" + 0.008*"two" + 0.007*"one" + 0.007*"also"')
(2, '0.015*"space" + 0.013*"equation" + 0.011*"theory" + 0.011*"function" + 0.010*"group" + 0.010*"solution" + 0.009*"operator" + 0.009*"result" + 0.007*"field" + 0.006*"case"')
(3, '0.021*"system" + 0.018*"control" + 0.012*"agent" + 0.012*"environment" + 0.010*"policy" + 0.010*"dynamic" + 0.010*"learning" + 0.009*"robot" + 0.008*"approach" + 0.007*"reinforcement"')
(4, '0.013*"data" + 0.010*"study" + 0.009*"research" + 0.009*"system" + 0.008*"analysis" + 0.007*"user" + 0.007*"model" + 0.006*"attack" + 0.006*"paper" + 0.005*"tool"')
(5, '0.012*"star" + 0.011*"mass" + 0.011*"galaxy" + 0.007*"black" + 0.007*"ho

## BERTopic
Pada pemodelan BERTopic sedikit berbeda, BERTopic lebih cangih dalam menangkap semantik pada kata sehingga teks utuk lah yang akan diaplikasikan pada model ini. Model embedding yang digunakan adalah "all-MiniLM-L6-v2".

In [15]:
!pip install bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 34.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [16]:
import torch

# Menggunakan GPU T4 dari google colab
if torch.cuda.is_available():
    print("GPU terdeteksi!")
    print(f"Nama GPU: {torch.cuda.get_device_name(0)}")
    device = "cuda"
else:
    # Jika tidak, beri peringatan
    print("GPU tidak ditemukan.")
    device = "cpu"

GPU terdeteksi!
Nama GPU: Tesla T4


In [17]:
"""Kode dibawah adalah kode untuk melakukan pemodelan dengan BERTopic."""

from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
import pandas as pd

# Pada tugas ini, model embedding yang digunakan adlaah "all-MiniLM-L6-v2"
embedding_model_name = "all-MiniLM-L6-v2"

# Memuat model ke GPU dengan SentenceTransformer
embedding_model = SentenceTransformer(embedding_model_name, device=device)

print(f"Embedding model '{embedding_model_name}' siap digunakan di {device}.")

# Inisiasi BERTopic
topic_model = BERTopic(
    embedding_model=embedding_model,
    language="english",
    calculate_probabilities=True,
    verbose=True, #
    min_topic_size=50
)

# Mempersiapkan dokumen dari kolom 'text'
docs_for_bertopic = df['text'].tolist()

# Training model BERTopic
topics, probs = topic_model.fit_transform(docs_for_bertopic)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2025-06-24 11:54:52,037 - BERTopic - Embedding - Transforming documents to embeddings.


Embedding model 'all-MiniLM-L6-v2' siap digunakan di cuda.


Batches:   0%|          | 0/3125 [00:00<?, ?it/s]

2025-06-24 11:59:28,680 - BERTopic - Embedding - Completed ✓
2025-06-24 11:59:28,684 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-06-24 12:01:59,364 - BERTopic - Dimensionality - Completed ✓
2025-06-24 12:01:59,386 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-06-24 12:04:41,491 - BERTopic - Cluster - Completed ✓
2025-06-24 12:04:41,569 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-06-24 12:04:54,918 - BERTopic - Representation - Completed ✓


In [18]:
# Menampilkan hasil BERTopic
print("\nRingkasan topik yang ditemukan oleh BERTopic:")
print(topic_model.get_topic_info())


Ringkasan topik yang ditemukan oleh BERTopic:
     Topic  Count                                              Name  \
0       -1  39809                                  -1_of_and_the_to   
1        0  15964                                  0_mass_the_of_at   
2        1   1677               1_segmentation_image_medical_images   
3        2   1646                     2_flow_fluid_particles_liquid   
4        3   1294                      3_speech_audio_speaker_music   
..     ...    ...                                               ...   
162    161     53           161_quantum_vqe_variational_eigensolver   
163    162     52  162_memristor_memristors_neuromorphic_memristive   
164    163     51                163_bergman_operators_spaces_hardy   
165    164     51        164_vae_latent_variational_representations   
166    165     51                     165_talking_facial_audio_face   

                                        Representation  \
0        [of, and, the, to, in, for, we, i